In [7]:
#This following Method will read a bunch of circuits and calculate the circuit depth of each of them lols.
from qiskit import *
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.h(1)
qc.x(0)
qc.cx(1,0)
qc.draw()

┌───┐     ┌───┐┌───┐
q_0: ┤ H ├──■──┤ X ├┤ X ├
     └───┘┌─┴─┐├───┤└─┬─┘
q_1: ─────┤ X ├┤ H ├──■──
          └───┘└───┘

In [4]:
#This obtains Circuit Depth:
def calc_depth(qc):
    return qc.depth()